### Check files dimension in the database

In [ ]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path
import os

st = study('Olfacto')
path_data = path.join(st.path,'database/Retrieval_By_Odor/') #Retrieval_EpiPerf_LowHigh
files = os.listdir(path_data) #Retrieval_By_Odor
#print(files)
for fi in files:
    if fi.endswith('bipo_sel_physFT.npz') and fi.startswith('LEFC'):
        loadname = path_data+fi
        mat = np.load(loadname)
        x = np.load(loadname)['x']
        print (fi, x.shape, mat.files)

## Remove bad electrodes after artefact detection
### TF with artefacts or 50Hz

In [2]:
import numpy as np
from brainpipe.system import study
from os import path
import mne

In [ ]:
st = study('respi')
pathfiles = path.join(st.path, 'database/')
phases = ['E1','E2']
rej_elec = {
    'CHAF' : [27,76,98,102,103,104],
    'VACJ' : [0,66,67,68,122,128], 
    'SEMC' : [36],
    'PIRJ' : [10,14,15,25,26,41,45,47,48,53,54,55,57,58,63,64,70,71,72,79,80,81,82,83,],
    'LEFC' : [47,48,49,50,140,141,],
    'MICP' : [4,9,13,18,19,20,71,],
    'FERJ': [],
            }
files = [k for k in st.search('_bipo.npz', folder=('database/'))]
for fi in files:
    mat = np.load(pathfiles+fi)
    x, sf, label, channel, xyz = mat['x'], mat['sf'], mat['label'], mat['channel'], mat['xyz']
    print (fi, x.shape, label.shape, channel.shape, xyz.shape)

    #Remove all bad electrodes from the database
    new_x = np.delete(x,rej_elec[fi[:4]], axis=0)
    new_label = np.delete(label,rej_elec[fi[:4]], axis=0)
    new_channel = np.delete(channel,rej_elec[fi[:4]], axis=0)
    new_xyz = np.delete(xyz,rej_elec[fi[:4]], axis=0)
    print (fi, len(rej_elec[fi[:4]]),new_x.shape, new_label.shape, new_channel.shape, new_xyz.shape)

    #Save all new information
    mat_new = {'x':new_x, 'label':new_label, 'channel':new_channel, 'xyz':new_xyz, 'sf':sf}
    file_source = fi.replace('bipo.npz','bipo2.npz')
    np.savez(pathfiles+file_source,**mat_new)

In [ ]:
st = study('respi')
pathfiles = path.join(st.path, 'database/')
phases = ['E1','E2']

files = [k for k in st.search('_bipo2.npz', folder=('database/'))]
for fi in files:
    mat = np.load(pathfiles+fi)
    x, sf, label, channel, xyz = mat['x'], mat['sf'], mat['label'], mat['channel'], mat['xyz']
    print (fi, x.shape, label.shape, channel.shape, xyz.shape)

    #Filter the data // Notch 50Hz
    raw_info = mne.create_info(list(channel), sf, ['seeg'] * len(channel))
    raw = mne.io.RawArray(x.squeeze(), raw_info, verbose=False)
    mne.io.RawArray.notch_filter(raw, freqs=np.arange(50, 251, 50),fir_design=None)
    raw_f = raw.get_data()[:,:,np.newaxis]
        
    #Save all new information
    mat_new = {'x':raw_f, 'label':label, 'channel':channel, 'xyz':xyz, 'sf':sf}
    file_source = fi.replace('bipo2.npz','bipo2_filt.npz')
    np.savez(pathfiles+file_source,**mat_new)

## Remove electrodes with WM

In [3]:
st = study('respi')
pathfiles = path.join(st.path, 'database/')

files = [k for k in st.search('bipo2_filt.npz', folder=('database/'))]
for fi in files:
    mat = np.load(pathfiles+fi)
    data, label, channel, xyz,sf= mat['x'], mat['label'], mat['channel'], mat['xyz'],mat['sf']
    print(fi, 'data shape: ', data.shape, label.shape, channel.shape, xyz.shape)

    #create a list of all electrodes to remove
    elecs = label.shape[0]
    elecs_rej = []
    for elec in range(elecs):
        #elec_rej_wm = np.where(label[elec].find('WM') != -1,int(elec),None)
        elec_rej_lcr = np.where(label[elec].find('LCR') != -1,int(elec),None)
        elec_rej_skull = np.where(label[elec].find('skull') != -1,int(elec),None)
        elec_rej_out = np.where(label[elec].find('out') != -1,int(elec),None)
        #if elec_rej_wm != None:
            #elecs_rej = np.append(elecs_rej,elec_rej_wm)
        if elec_rej_lcr != None :
            elecs_rej = np.append(elecs_rej,elec_rej_lcr)
        if elec_rej_skull != None and elec_rej_skull != elec_rej_lcr:
            elecs_rej = np.append(elecs_rej,elec_rej_skull)
        if elec_rej_out != None and elec_rej_out != elec_rej_skull  and elec_rej_out != elec_rej_lcr:
            elecs_rej = np.append(elecs_rej, elec_rej_out)
    print(fi[:4], elecs, elecs_rej)

    #Remove all bad electrodes from the database
    new_data, new_label = np.delete(data,elecs_rej, axis=0), np.delete(label,elecs_rej, axis=0)
    new_channel, new_xyz = np.delete(channel,elecs_rej, axis=0), np.delete(xyz,elecs_rej, axis=0)
    print (fi,'new data',new_data.shape, new_label.shape,new_channel.shape, new_xyz.shape)

    #Save all new information
    mat_new = {'x':new_data, 'label':new_label, 'channel':new_channel, 'xyz':new_xyz, 'sf':sf}
    filename2 = fi.replace('bipo2_filt.npz','bipo3.npz')
    np.savez(pathfiles+filename2,**mat_new)
    del elecs, elecs_rej, new_data, new_label, new_channel, new_xyz

-> respi loaded
CHAF_E1_sigs_bipo2_filt.npz data shape:  (96, 529888, 1) (96,) (96,) (96, 3)
CHAF 96 []
CHAF_E1_sigs_bipo2_filt.npz new data (96, 529888, 1) (96,) (96,) (96, 3)
CHAF_E2_sigs_bipo2_filt.npz data shape:  (96, 336192, 1) (96,) (96,) (96, 3)
CHAF 96 []
CHAF_E2_sigs_bipo2_filt.npz new data (96, 336192, 1) (96,) (96,) (96, 3)
MICP_E1_sigs_bipo2_filt.npz data shape:  (96, 543344, 1) (96,) (96,) (96, 3)
MICP 96 []
MICP_E1_sigs_bipo2_filt.npz new data (96, 543344, 1) (96,) (96,) (96, 3)
MICP_E2_sigs_bipo2_filt.npz data shape:  (96, 437568, 1) (96,) (96,) (96, 3)
MICP 96 []
MICP_E2_sigs_bipo2_filt.npz new data (96, 437568, 1) (96,) (96,) (96, 3)
PIRJ_E1_sigs_bipo2_filt.npz data shape:  (83, 522528, 1) (83,) (83,) (83, 3)
PIRJ 83 []
PIRJ_E1_sigs_bipo2_filt.npz new data (83, 522528, 1) (83,) (83,) (83, 3)
PIRJ_E2_sigs_bipo2_filt.npz data shape:  (83, 445024, 1) (83,) (83,) (83, 3)
PIRJ 83 []
PIRJ_E2_sigs_bipo2_filt.npz new data (83, 445024, 1) (83,) (83,) (83, 3)
SEMC_E1_sigs_bipo2

### Add AAL & BA labels to subjects' matrices
### Add also the Mai labels (Jane-AL) post fMRI

In [4]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path

st = study('respi')
path_data = path.join(st.path,'database/')
path_labels = path.join(path_data,'_labels/') #create with visbrain in 4_Visbrain new scripts

files = [k for k in st.search('_bipo3.npz', folder=('database/'))]
for fi in files:
    mat = np.load(path_data+fi)
    print(mat.files)
    kwargs = {}
    kwargs['x'], kwargs['channel'], kwargs['xyz'] = mat['x'], mat['channel'],mat['xyz']
    kwargs['label'], kwargs['sf']=mat['label'],mat['sf']
    # Load the AAL labels
    aal = 'Elecs_'+fi[:4]+'_labels_aal.csv'
    df = pd.read_csv(path_labels+aal, sep=',')
    kwargs['aal'] = df['aal'].values
    # Load the BA labels
    BA = 'Elecs_'+fi[:4]+'_labels_brodmann.csv'
    df = pd.read_csv(path_labels+BA, sep=',')
    kwargs['BA'] = df['brodmann'].values
    # Update the datafile
    filename2 = fi.replace('bipo3.npz','bipo3_phys.npz')
    np.savez(path_data+filename2, **kwargs)
    mat2 = np.load(path_data+filename2)
    print(mat2.files)
    print(fi[:4],mat2['x'].shape, mat2['xyz'].shape, mat2['aal'].shape, mat2['BA'].shape)

-> respi loaded
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
CHAF (96, 529888, 1) (96, 3) (96,) (96,)
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
CHAF (96, 336192, 1) (96, 3) (96,) (96,)
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
MICP (96, 543344, 1) (96, 3) (96,) (96,)
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
MICP (96, 437568, 1) (96, 3) (96,) (96,)
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
PIRJ (83, 522528, 1) (83, 3) (83,) (83,)
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
PIRJ (83, 445024, 1) (83, 3) (83,) (83,)
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
SEMC (91, 424560, 1) (91, 3) (91,) (91,)
['x', 'channel', 'label', 'xyz', 'sf']
['x', 'BA', 'label', 

In [5]:
import pandas as pd
import numpy as np
from brainpipe.system import study
from os import path

st = study('respi')
path_data = path.join(st.path,'database/')
path_labels = path.join(path_data,'_labels/') #create with visbrain in 4_Visbrain new scripts

files = [k for k in st.search('_bipo3_phys.npz', folder=('database/'))]
for fi in files:
    mat = np.load(path_data+fi)
    print(mat.files)
    kwargs = {}
    kwargs['x'], kwargs['channels'], kwargs['xyz'] = mat['x'], mat['channel'],mat['xyz']
    kwargs['labels'], kwargs['sf']=mat['label'],mat['sf']
    # Load the AAL labels
    aal = 'Elecs_'+fi[:4]+'_labels_aal.csv'
    df = pd.read_csv(path_labels+aal, sep=',')
    kwargs['aal'] = df['aal'].values
    # Load the BA labels
    BA = 'Elecs_'+fi[:4]+'_labels_brodmann.csv'
    df = pd.read_csv(path_labels+BA, sep=',')
    kwargs['BA'] = df['brodmann'].values
    #Load the new_labels from Mai atlas
    Mai_file = '0_all_subjects_info_elecs.csv'
    df2 = pd.read_csv(path_labels+Mai_file, sep=',')
    Mai_data = df2['Mai_RL'].loc[df2['subjects'] == fi[:4]].values
    kwargs['Mai_RL'] = Mai_data
    #Add  the new labels Mai RL
    kwargs['Mai'] = Mai_data+'_'+['R' if x > 0 else 'L' for x in df2['x'].loc[df2['subjects'] == fi[:4]].values]
    # Update the datafile
    filename2 = fi.replace('bipo3_phys.npz','bipo3_phys.npz')
    np.savez(path_data+filename2, **kwargs)
    mat2 = np.load(path_data+filename2)
    print(mat2.files, mat2['Mai'][:5],mat2['Mai_RL'][:5])
    print(fi[:4],mat2['x'].shape, mat2['xyz'].shape, mat2['aal'].shape, mat2['BA'].shape,
          mat2['Mai'].shape, mat2['Mai_RL'].shape)

-> respi loaded
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
['x', 'labels', 'channels', 'BA', 'aal', 'xyz', 'Mai', 'Mai_RL', 'sf'] ['PCC_L' 'PCC_L' 'PrG_L' 'PrG_L' 'PrG_L'] ['PCC' 'PCC' 'PrG' 'PrG' 'PrG']
CHAF (96, 529888, 1) (96, 3) (96,) (96,) (96,) (96,)
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
['x', 'labels', 'channels', 'BA', 'aal', 'xyz', 'Mai', 'Mai_RL', 'sf'] ['PCC_L' 'PCC_L' 'PrG_L' 'PrG_L' 'PrG_L'] ['PCC' 'PCC' 'PrG' 'PrG' 'PrG']
CHAF (96, 336192, 1) (96, 3) (96,) (96,) (96,) (96,)
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
['x', 'labels', 'channels', 'BA', 'aal', 'xyz', 'Mai', 'Mai_RL', 'sf'] ['Amg_L' 'Amg_L' 'Amg_L' 'Amg_L' 'MTG_L'] ['Amg' 'Amg' 'Amg' 'Amg' 'MTG']
MICP (96, 543344, 1) (96, 3) (96,) (96,) (96,) (96,)
['x', 'BA', 'label', 'xyz', 'channel', 'sf', 'aal']
['x', 'labels', 'channels', 'BA', 'aal', 'xyz', 'Mai', 'Mai_RL', 'sf'] ['Amg_L' 'Amg_L' 'Amg_L' 'Amg_L' 'MTG_L'] ['Amg' 'Amg' 'Amg' 'Amg' 'MTG']
MICP (96, 437568, 1) (96, 3) (96,) (9